# Invoke Bedrock model for text generation using zero-shot prompt

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

In this notebook we show you how to use a LLM to generate an email response to a customer who provided negative feedback on the quality of customer service that they received from the support engineer. 

We will use Meta's Llama 2 large model using the Boto3 API. 

The prompt used in this example is called a zero-shot prompt because we are not providing any examples of text alongside their classification other than the prompt.

**Note:** *This notebook can be run within or outside of AWS environment.*

#### Context
To demonstrate the text generation capability of Amazon Bedrock, we will explore the use of Boto3 client to communicate with Amazon Bedrock API. We will demonstrate different configurations available as well as how simple input can lead to desired outputs.

#### Pattern
We will simply provide the Amazon Bedrock API with an input consisting of a task, an instruction and an input for the model under the hood to generate an output without providing any additional example. The purpose here is to demonstrate how the powerful LLMs easily understand the task at hand and generate compelling outputs.

![](./images/bedrock.jpg)

#### Use case
To demonstrate the generation capability of models in Amazon Bedrock, let's take the use case of email generation.

#### Persona
You are Bob a Customer Service Manager at AnyCompany and some of your customers are not happy with the customer service and are providing negative feedbacks on the service provided by customer support engineers. Now, you would like to respond to those customers humbly aplogizing for the poor service and regain trust. You need the help of an LLM to generate a bulk of emails for you which are human friendly and personalized to the customer's sentiment from previous email correspondence.

#### Implementation
To fulfill this use case, in this notebook we will show how to generate an email with a thank you note based on the customer's previous email.We will use the Meta Llama 2 Text Large model using the Amazon Bedrock API with Boto3 client. 

## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you've run the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb#Prerequisites) notebook. ⚠️ ⚠️ ⚠️


In [5]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Generate text

Following on the use case explained above, let's prepare an input for  the Amazon Bedrock service to generate an email.

In [17]:
# create the prompt
prompt_data = """
Command: Write an email from Bob, Customer Service Manager, to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""


Let's start by using the Meta's Llama 2 Large model. Meta Llama 2 Large supports a context window of ~4k tokens and accepts the following parameters:
- `inputText`: Prompt to the LLM
- `textGenerationConfig`: These are the parameters that model will take into account while generating the output.

In [3]:
body = json.dumps({ 
	'prompt': prompt_data,
    'max_gen_len': 512,
	'top_p': 0.9,
	'temperature': 0.2
})

The Amazon Bedrock API provides you with an API `invoke_model` which accepts the following:
- `modelId`: This is the model ARN for the various foundation models available under Amazon Bedrock
- `accept`: The type of input request
- `contentType`: The content type of the output
- `body`: A json string consisting of the prompt and the configurations

Check [documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) for Available text generation model Ids

#### Invoke the Meta's Llama 2 Large language model

First, we explore how the model generates an output based on the prompt created earlier.

##### Complete Output Generation

In [18]:
modelId = 'meta.llama2-13b-chat-v1' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"
try:

    response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read().decode('utf-8'))

    outputText = response_body['generation'].strip()

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

In [19]:
# The relevant portion of the response begins after the first newline character
# Below we print the response beginning after the first occurence of '\n'.

email = outputText[outputText.index('\n')+1:]
print_ww(email)



Subject: Apology and Resolution for Poor Service Experience

Dear John,

I am writing to express my sincerest apologies for the poor service experience you
received from our customer support engineer, Jane Smith. We take all feedback
seriously and are committed to providing the highest level of service to our
customers.

I have reviewed the details of your case and I want to assure you that we are taking
appropriate actions to address the issues you encountered. Our customer support
engineers are trained to provide professional and timely assistance, and it is
disappointing to learn that Jane did not meet these expectations.

To make up for the inconvenience and frustration caused, I would like to offer you
a complimentary one-month subscription to our premium support service. This
service includes priority support, dedicated account management, and access to
our advanced technical resources. We hope that this gesture will demonstrate our
commitment to your satisfaction and loyalty as

##### Streaming Output Generation
Above is an example email generated by the Meta Llama 2 Large model by understanding the input request and using its inherent understanding of the different modalities. This request to the API is synchronous and waits for the entire output to be generated by the model.

Bedrock also supports that the output can be streamed as it is generated by the model in form of chunks. Below is an example of invoking the model with streaming option. `invoke_model_with_response_stream` returns a `ResponseStream` which you can read from.

In [24]:
output = []
try:
    
    response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
    stream = response.get('body')
    
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['generation']
                output.append(text)
                print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
                i+=1
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

		**Chunk 1**
,

		**Chunk 2**
 James

		**Chunk 3**
.

		**Chunk 4**



		**Chunk 5**



		**Chunk 6**
Subject

		**Chunk 7**
:

		**Chunk 8**
 Ap

		**Chunk 9**
ology

		**Chunk 10**
 and

		**Chunk 11**
 Resol

		**Chunk 12**
ution

		**Chunk 13**
 for

		**Chunk 14**
 Po

		**Chunk 15**
or

		**Chunk 16**
 Service

		**Chunk 17**
 Exper

		**Chunk 18**
ience

		**Chunk 19**



		**Chunk 20**



		**Chunk 21**
D

		**Chunk 22**
ear

		**Chunk 23**
 John

		**Chunk 24**
,

		**Chunk 25**



		**Chunk 26**



		**Chunk 27**
I

		**Chunk 28**
 am

		**Chunk 29**
 writing

		**Chunk 30**
 to

		**Chunk 31**
 apolog

		**Chunk 32**
ize

		**Chunk 33**
 for

		**Chunk 34**
 the

		**Chunk 35**
 poor

		**Chunk 36**
 service

		**Chunk 37**
 experience

		**Chunk 38**
 you

		**Chunk 39**
 recently

		**Chunk 40**
 had

		**Chunk 41**
 with

		**Chunk 42**
 our

		**Chunk 43**
 customer

		**Chunk 44**
 support

		**Chunk 45**
 engineer

		**Chunk 46**
,

		**Chunk 47**
 James

		**Chunk 4

The above helps to quickly get output of the model and let the service complete it as you read. This assists in use-cases where there are longer pieces of text that you request the model to generate. You can later combine all the chunks generated to form the complete output and use it for your use-case

In [36]:
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
outputText = ''.join(output)
email = outputText[outputText.index('\n')+1:]
print(email)

		**COMPLETE OUTPUT**


Subject: Apology and Resolution for Poor Service Experience

Dear John,

I am writing to apologize for the poor service experience you recently had with our customer support engineer, James. We take all feedback seriously and are committed to providing the highest level of service to our customers.

I have reviewed the details of your case and I want to assure you that we are taking immediate action to address the issues you encountered. Our team is working diligently to resolve the problems you faced and to prevent them from happening again in the future.

As a token of our commitment to your satisfaction, I would like to offer you a complimentary one-month subscription to our premium support service. This service includes priority support, dedicated account management, and access to our advanced technical support team.

Please let me know if there is anything else we can do to resolve your issue and improve your experience with our company. Your satisfaction i

## Conclusion
You have now experimented with using `boto3` SDK which provides a vanilla exposure to Amazon Bedrock API. Using this API you have seen the use case of generating an email responding to a customer due to their negative feedback.

### Take aways
- Adapt this notebook to experiment with different models available through Amazon Bedrock such as Anthropic Claude and AI21 Labs Jurassic models.
- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the token length to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

## Thank You